<a href="https://colab.research.google.com/github/mantoan-thi/Day_trade_mini_indice/blob/main/Carteira_de_ativos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
from pandas_datareader import data as wb
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
def baixar_ativo(ativo,data_inicial,data_final):
    preco = pd.DataFrame()
    for t in ativo:
        preco[t] = wb.DataReader(t,data_source='yahoo',start=data_inicial,end=data_final)['Adj Close']
    return preco
def returno_anual(carteira):
    preco_normalizado = carteira/carteira.iloc[0]*100
    # Retorno diário log
    retorno_log = np.log(preco_normalizado/preco_normalizado.shift(1))
    return retorno_log

In [ ]:
ativos = pd.read_csv("C:/Users/Thiago/Downloads/IBOVDia_07-06-21.csv",engine='python',delimiter=";")
ativos.reset_index(inplace=True)
ativos.drop(columns = ativos.columns[-1], inplace=True)
ativos.dropna(inplace=True)
ativos['Part. (%)'] = ativos['Part. (%)'].apply(lambda x: x.replace(",","."))   
ativos['Part. (%)'] = ativos['Part. (%)'].apply(float) 
ativos = ativos[ativos['Part. (%)'] > 4]

In [ ]:
indice_ibov=['^BVSP']
New_ativos = []
for i in ativos['Código']:
    New_ativos.append(i +'.SA')
New_ativos = New_ativos+indice_ibov
New_ativos

['B3SA3.SA',
 'BBDC4.SA',
 'ITUB4.SA',
 'PETR3.SA',
 'PETR4.SA',
 'VALE3.SA',
 '^BVSP']

In [ ]:
#New_ativos = ['B3SA3.SA','GNDI3.SA','MGLU3.SA','WEGE3.SA']
dataset = baixar_ativo(New_ativos,'2020-01-01','2021-06-05')
retorno_anual = returno_anual(dataset)

In [ ]:
def analise(num_comb):
    melhor_retorno =0.2
    melhores_pesos = []
    print('Analisando...')
    print('Total de análises previstas: ',num_comb)
    for x in range(num_comb):
        print(x,end='\r')
        pesos = np.random.random(len(New_ativos))
        pesos/= np.sum(pesos)
        ret = np.sum(pesos*retorno_anual.mean())*250
        volt = np.sqrt(np.dot(pesos.T,np.dot(retorno_anual.cov()*250,pesos)))
        if ret > melhor_retorno:
            melhor_retorno = ret
            Retorno_da_carteira = ret
            Volatilidade_da_carteira = volt
            melhores_pesos = pesos
        #Retorno_da_carteira.append(np.sum(pesos*retorno_anual.mean())*250)
        #Volatilidade_da_carteira.append(np.sqrt(np.dot(pesos.T,np.dot(retorno_anual.cov()*250,pesos))))
    print('\n\nMelhor retorno: {:0.2%}'.format(round(Retorno_da_carteira,2)))
    print('Volatilidade: {:0.2%}'.format(round(Volatilidade_da_carteira,2)))
    print('Melhores pesos: ',melhores_pesos) 

    print('\nComposição da carteira')
    count = 0
    valor = 1000
    for i in dataset.columns:
        print(i,'= {:0.2%}'.format(round(melhores_pesos[count],2)))
        count +=1
    valor = 89.30   
    print('\nInvestimento',valor)   
    count = 0
    for i in dataset.columns:
        vlr_invs = round(melhores_pesos[count],2)*valor
        if vlr_invs > 0:
            print('valor investido: ',i,'= R$',(round(melhores_pesos[count]*valor,2)))
        count +=1


In [ ]:
dataset = baixar_ativo(New_ativos,'2020-01-01','2021-06-05')

In [ ]:
dataset

,B3SA3.SA,BBDC4.SA,ITUB4.SA,PETR3.SA,PETR4.SA,VALE3.SA,^BVSP
Date,,,,,,,
2020-01-02,14.351929,33.072002,36.495586,30.542723,29.698208,49.901928,118573.0
2020-01-03,13.938331,33.087223,36.111732,29.788467,29.456367,49.534527,117707.0
2020-01-06,13.804705,32.496223,35.574322,30.756895,29.804619,49.240608,116878.0
2020-01-07,14.250122,31.931686,34.749016,30.337868,29.688534,49.598820,116662.0
2020-01-08,14.211944,31.437706,34.182827,29.844337,29.504732,49.608006,116247.0
...,...,...,...,...,...,...,...
2021-05-28,17.480000,26.521406,29.614954,26.709999,26.990000,111.589996,125561.0
2021-05-31,17.540001,26.461447,29.525002,26.650000,26.870001,114.779999,126216.0
2021-06-01,17.700001,27.061026,30.600000,27.230000,27.290001,113.199997,128267.0


In [ ]:
for i in dataset.columns:
    print('Ativos: ',i,' = ',dataset[i][-4:-3].values)

Ativos:  ABEV3.SA  =  [17.95000076]
Ativos:  ASAI3.SA  =  [89.44999695]
Ativos:  AZUL4.SA  =  [43.]
Ativos:  BTOW3.SA  =  [59.75]
Ativos:  B3SA3.SA  =  [17.54000092]
Ativos:  BIDI11.SA  =  [68.33000183]
Ativos:  BBSE3.SA  =  [23.39999962]
Ativos:  BRML3.SA  =  [11.18000031]
Ativos:  BBDC3.SA  =  [22.84305573]
Ativos:  BBDC4.SA  =  [26.46144676]
Ativos:  BRAP4.SA  =  [71.20999908]
Ativos:  BBAS3.SA  =  [33.49000168]
Ativos:  BRKM5.SA  =  [50.06999969]
Ativos:  BRFS3.SA  =  [25.76000023]
Ativos:  BPAC11.SA  =  [124.05000305]
Ativos:  CRFB3.SA  =  [22.39999962]
Ativos:  CCRO3.SA  =  [13.77999973]
Ativos:  CMIG4.SA  =  [13.57999992]
Ativos:  HGTX3.SA  =  [32.90000153]
Ativos:  CIEL3.SA  =  [4.21000004]
Ativos:  COGN3.SA  =  [4.21999979]
Ativos:  CPLE6.SA  =  [6.38000011]
Ativos:  CSAN3.SA  =  [23.37000084]
Ativos:  CPFE3.SA  =  [28.70999908]
Ativos:  CVCB3.SA  =  [24.88999939]
Ativos:  CYRE3.SA  =  [23.89999962]
Ativos:  ECOR3.SA  =  [12.48999977]
Ativos:  ELET3.SA  =  [43.84000015]
Ativos